In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Input, Reshape, Conv2D, ReLU, MaxPool2D, Masking
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import ast
from tensorflow.keras.optimizers import Adam

# Load and preprocess the data
# Replace 'train.csv' with the actual path to your dataset
data = pd.read_csv('dataset/train.csv', header = None, converters = {
    3: ast.literal_eval,
    4: ast.literal_eval,
    5: ast.literal_eval
}, skiprows = 1)

df = pd.DataFrame()

df['acc_x'] = data[3]
df['acc_y'] = data[4]
df['acc_z'] = data[5]
df['gesture'] = data[2]

#remove invalid rows
df.drop(df.loc[df['acc_x']==0].index, inplace=True)
df.drop(df.loc[df['acc_y']==0].index, inplace=True)
df.drop(df.loc[df['acc_z']==0].index, inplace=True)

df = df.dropna()

# Convert the lists into arrays
acc_x = df['acc_x'].values
acc_y = df['acc_y'].values
acc_z = df['acc_z'].values

# Combine all axes into a sequence of shape (timesteps, features)
sequences = [np.array([x, y, z]).T for x, y, z in zip(acc_x, acc_y, acc_z)]

# Pad sequences to the length of the longest sequence
padded_sequences = pad_sequences(sequences, maxlen = 32, padding='post', dtype='float32')

# Encode labels
labels = df['gesture'].values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)


2025-01-04 15:39:10.953633: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-04 15:39:10.975307: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-04 15:39:10.975330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-04 15:39:10.975966: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-04 15:39:10.980299: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#DATA AUGMENTATION preprocessing

#adding noise
def add_noise(data, noise_level=0.05):
    return (data + np.random.normal(0, noise_level, data.shape)).astype(np.float32)

# Original data: `x_train` (accelerometer sequences), `y_train` (labels)

padded_sequences = np.concatenate((padded_sequences, add_noise(padded_sequences)))
categorical_labels = np.concatenate((categorical_labels, categorical_labels))




def scale_data(data, scaling_factor=0.1):
    """
    Scale the data by a random factor.
    Args:
        data: Numpy array of shape (time_steps, 3).
        scaling_factor: Max scaling factor variation.
    Returns:
        Scaled data.
    """
    factor = 1 + np.random.uniform(-scaling_factor, scaling_factor)
    return (data * factor).astype(np.float32)

padded_sequences = np.concatenate((padded_sequences, scale_data(padded_sequences)))
categorical_labels = np.concatenate((categorical_labels, categorical_labels))





In [3]:
# Split the data
X_train, X_validation, y_train, y_validation = train_test_split(
    padded_sequences, categorical_labels, test_size=0.2, random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42)


In [4]:
# Length to standardize data to.
# Average is 19.708 samples, 
# max is 51, min is 10
data_truncate_pad_length = 32

# Number of epochs per train iteration.
epochs = 15

# Number of iterations to simulate over.
num_iterations = 5

# Number of gestures in the dataset.
num_gestures = 20

# Number of total users in the training file.
num_train_users = 7

# Create an array of the zero-based gesture IDs.
classes = np.arange( num_gestures )

In [5]:
def create_lstm_model( num_classes=20,
                       dropout=0.8, units=128,
                       data_length=19,
                       optimizer=Adam( learning_rate=0.001 )
                     ):
    """
    Create a simple bidirectional LSTM model for classification. Creates a
    bidirectional LSTM layer, dropout, and two dense layers.

    :param num_classes: Number of classes.

    :param dropout: Dropout rate

    :param units: Number of units in LSTM layer.

    :param data_length: Length of time data for LSTM layer input.

    :param optimizer: Optimizer to use for model compilation.

    :return: The compiled model with LSTM, dropout, and two dense layers.
    """

    # Create the model object.
    model = tf.keras.models.Sequential()

    # Add an LSTM layer.
    # Input size is (data_length,3):
    #   data_length time samples from data.
    #   3 dimensions (x, y, z accelerometer data).
    model.add(
        tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM( units=units, input_shape=[data_length, 3] )
        )
    )

    # Add dropout layer to reduce overfitting.
    model.add( tf.keras.layers.Dropout( rate=dropout ) )

    # Add final dense layers.
    model.add( tf.keras.layers.Dense( units=16, activation='relu' ) )
    model.add( tf.keras.layers.Dense( units=num_classes,
                                      activation='softmax' ) )

    model.compile( loss='categorical_crossentropy', optimizer=optimizer,
                   metrics=['accuracy'] )

    return model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

# Layer parameters.
dropout_rate = 0.0
lstm_units = 32


# Optimizer parameters.
lr = 0.001
model_optimizer = Adam( learning_rate=lr,)


# Callback parameters.
monitor = 'loss'
min_delta = 0.001
patience = 20
earlystop_callback = EarlyStopping( monitor=monitor, min_delta=min_delta,
                                    verbose=1, patience=patience,
                                    restore_best_weights=True )

# lstm_callbacks = [earlystop_callback]
lstm_callbacks = None
fit_params = ( epochs, lstm_callbacks )


model = create_lstm_model( num_classes=20,
                               dropout=dropout_rate,
                               units=lstm_units,
                               data_length=data_truncate_pad_length,
                               optimizer=model_optimizer )

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_validation, y_validation),
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
214/214 [==============================] - 3s 8ms/step - loss: 2.1977 - accuracy: 0.2589 - val_loss: 1.3321 - val_accuracy: 0.6101
Epoch 2/50
214/214 [==============================] - 1s 6ms/step - loss: 0.8748 - accuracy: 0.7513 - val_loss: 0.6222 - val_accuracy: 0.8145
Epoch 3/50
214/214 [==============================] - 1s 6ms/step - loss: 0.4391 - accuracy: 0.8670 - val_loss: 0.3859 - val_accuracy: 0.8654
Epoch 4/50
214/214 [==============================] - 1s 6ms/step - loss: 0.2797 - accuracy: 0.9152 - val_loss: 0.2400 - val_accuracy: 0.9338
Epoch 5/50
214/214 [==============================] - 1s 6ms/step - loss: 0.1902 - accuracy: 0.9526 - val_loss: 0.1555 - val_accuracy: 0.9671
Epoch 6/50
214/214 [==============================] - 1s 6ms/step - loss: 0.1215 - accuracy: 0.9713 - val_loss: 0.1337 - val_accuracy: 0.9693
Epoch 7/50
214/214 [==============================] - 1s 6ms/step - loss: 0.0924 - accuracy: 0.9803 - val_loss: 0.1087 - val_accuracy: 0.9719
Epoch 

In [7]:

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the model
model.save('lstm_gesture_classification_model.h5')

# Decode predicted labels for interpretability
predicted_classes = label_encoder.inverse_transform(np.argmax(model.predict(X_test), axis=1))


72/72 [==============================] - 0s 2ms/step - loss: 0.0086 - accuracy: 0.9965
Test Loss: 0.008621175773441792, Test Accuracy: 0.9964912533760071


/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


72/72 [==============================] - 0s 2ms/step


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 64)                9216      
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 20)                340       
                                                                 
Total params: 10596 (41.39 KB)
Trainable params: 10596 (41.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(padded_sequences).batch(1).take(32):
    yield [input_value]
       
# Ensure input shape is fixed
model.build(input_shape=(None, 32, 3))  # Example: Fixed length 32 timesteps, 3 features

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Apply integer quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Ensure compatibility
converter.inference_input_type = tf.uint8  # Optional: Set input type
converter.inference_output_type = tf.uint8  # Optional: Set output type

tflite_model = converter.convert()

# Save the model
with open('lstm_gesture_classification_model_integer.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model with full integer quantization saved as gesture_classification_model_integer.tflite")

INFO:tensorflow:Assets written to: /tmp/tmp6ocr_6he/assets


INFO:tensorflow:Assets written to: /tmp/tmp6ocr_6he/assets
/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-01-02 16:16:14.558728: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-01-02 16:16:14.558753: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-01-02 16:16:14.559057: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp6ocr_6he
2025-01-02 16:16:14.571596: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-01-02 16:16:14.571620: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp6ocr_6he
2025-01-02 16:16:14.600540: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR

ConverterError: /home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/runpy.py:196:1: error: 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
    return _run_code(code, main_globals, None,
^
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/runpy.py:196:1: error: failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
    return _run_code(code, main_globals, None,
^
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: Lowering tensor list ops is failed. Please consider using Select TF ops and disabling `_experimental_lower_tensor_list_ops` flag in the TFLite converter object. For example, converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]\n converter._experimental_lower_tensor_list

In [10]:
import os

# Convert the model to TFLite without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
fp_tflite_model = converter.convert()

# Save the model to disk
open("lstm_model.tflite", "wb").write(fp_tflite_model)

# Show the model size for the non-quantized HDF5 model
fp_h5_in_kb = os.path.getsize('models/gesture_classification_model.h5') / 1024
print("HDF5 Model size without quantization: %d KB" % fp_h5_in_kb)

# Show the model size for the non-quantized TFLite model
fp_tflite_in_kb = os.path.getsize('cnn_model_f32.tflite') / 1024
print("TFLite Model size without quantization: %d KB" % fp_tflite_in_kb)

# Determine the reduction in model size
print("\nReduction in file size by a factor of %f" % (fp_h5_in_kb / fp_tflite_in_kb))

INFO:tensorflow:Assets written to: /tmp/tmp8_n27ma6/assets


INFO:tensorflow:Assets written to: /tmp/tmp8_n27ma6/assets
2025-01-02 16:16:21.927245: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-01-02 16:16:21.927266: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-01-02 16:16:21.927376: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp8_n27ma6
2025-01-02 16:16:21.939439: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-01-02 16:16:21.939477: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp8_n27ma6
2025-01-02 16:16:21.979069: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-01-02 16:16:22.067929: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp8_n27ma6
2025-01-02 16:16:22.125666: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

ConverterError: /home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/runpy.py:196:1: error: 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
    return _run_code(code, main_globals, None,
^
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/runpy.py:196:1: error: failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
    return _run_code(code, main_globals, None,
^
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: Lowering tensor list ops is failed. Please consider using Select TF ops and disabling `_experimental_lower_tensor_list_ops` flag in the TFLite converter object. For example, converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]\n converter._experimental_lower_tensor_list

In [9]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

# Show the model size for the 8-bit quantized TFLite model
tflite_quant_in_kb = os.path.getsize('models/cnn_gesture_classification_model_integer.tflite') / 1024
print("TFLite Model size with 8-bit quantization: %d KB" % tflite_quant_in_kb)


input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
TFLite Model size with 8-bit quantization: 14 KB


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
# Helper function to run inference on a TFLite model

test_sequence_indices = range(X_test.shape[0])

print(y_test)

def run_tflite_model(tflite_file, test_image_indices):
  global X_test

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = []
  for i, test_sequence_index in enumerate(test_sequence_indices):
    test_sequence = X_test[test_sequence_index]
    test_label = y_test[test_sequence_index]

    if (test_sequence_index % 100 == 0):
      print("Evaluated on %d sequences." % test_sequence_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_sequence = test_sequence / input_scale + input_zero_point

    test_sequence = np.expand_dims(test_sequence, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_sequence)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions.append(np.argmax(output, axis=-1))

  return predictions


# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global X_test
  global y_test

  test_sequence_indices = range(X_train.shape[0])
  predictions = run_tflite_model(tflite_file, test_sequence_indices)
    
  labels = np.argmax(y_test, axis=1)

  accuracy = (np.sum(labels == predictions) * 100) / len(X_test)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(X_test)))

    


[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
import pathlib
# this might take a few minutes (~ 1- 2 minutes)
# if it takes longer than that, I suggest to 
# restart the runtime and try again
# if the issue still persists, restart your computer
tflite_model_quant_int8_file = pathlib.Path('models/cnn_gesture_classification_model_integer.tflite')
tflite_model_quant_int8_model_type = "Full Post-Quantized INT8"

evaluate_model(tflite_model_quant_int8_file, tflite_model_quant_int8_model_type)

input_details = interpreter.get_input_details()[0]
input_scale, input_zero_point = input_details["quantization"]

print(input_scale)
print(input_zero_point)




Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full Post-Quantized INT8 model accuracy is 95.9211% (Number of test samples=2280)
0.026128482073545456
119


In [12]:
# we plot also the confusion matrix of the quantized model
tflite_model_quant_int8_pred = run_tflite_model(tflite_model_quant_int8_file, range(y_test.shape[0]))

# compute the accuracy of the quantized model
from sklearn.metrics import accuracy_score

labels = np.argmax(y_test, axis=1)

full_int8_accuracy = accuracy_score(labels, tflite_model_quant_int8_pred)
print("Full-precision model accuracy is %.4f%% (Number of test samples=%d)" % (test_accuracy * 100, len(y_test)))
print("Quantized model accuracy is %.4f%% (Number of test samples=%d)" % (full_int8_accuracy * 100, len(y_test)))

Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full-precision model accuracy is 97.5000% (Number of test samples=2280)
Quantized model accuracy is 95.9211% (Number of test samples=2280)


In [13]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [14]:
c_model_name = 'q8_cnn'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [15]:
input_details = interpreter.get_input_details()[0]
input_scale, input_zero_point = input_details["quantization"]

# save the test data as numpy arrays
np.save('x_test_gestures.npy', (X_test / input_scale + input_zero_point).astype(np.uint8))
np.save('y_test_gestures.npy', (y_test.astype(np.uint8)))

# print the location of the files
print('Test image data location: ', os.path.abspath('x_test_gestures.npy'))
print('Test labels location: ', os.path.abspath('y_test_gestures.npy'))

Test image data location:  /home/amroset/Machine Learning on Microcontrollers/Project/x_test_gestures.npy
Test labels location:  /home/amroset/Machine Learning on Microcontrollers/Project/y_test_gestures.npy


In [16]:
X_test_try = (X_test[14] / input_scale + input_zero_point).astype(np.uint8)

print(input_scale)
print(input_zero_point)
print(X_test[19])
print(np.argmax(y_test[14]))

0.026128482073545456
119
[[-1.2017318   0.16807531  0.15167339]
 [-1.2530056   0.16807531  0.15167339]
 [-1.1504579   0.16807531  0.15167339]
 [-1.1504579   0.20704933  0.28464743]
 [-0.27880186  1.3762691   0.7832997 ]
 [-0.02243226  2.5844631   0.1849169 ]
 [ 0.38775876 -1.6637025  -0.11427449]
 [-1.6631967  -1.8585724  -1.7432058 ]
 [ 0.02884166  0.20704933 -2.2086146 ]
 [ 2.0285232  -0.10474256  2.3789873 ]
 [ 0.38775876 -0.6893526   1.1157348 ]
 [ 0.7979498  -0.6503786  -0.04778747]
 [ 0.95177156 -0.10474256 -0.08103098]
 [ 0.69540197  0.3239714  -0.4467094 ]
 [ 0.6441284  -0.06576855 -0.24724855]
 [ 0.7979498  -0.06576855 -0.31373534]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]


In [17]:
def write_test_sample_to_header(X_test_try, output_file):
    """
    Extract a test sample from X_test and write it to a data.h file.
    
    Args:
        X_test (np.ndarray): The test dataset.
        sample_index (int): Index of the sample to extract.
        output_file (str): Path to the output .h file.
    """
    
    # Flatten the sample if it is multi-dimensional
    flattened_sample = X_test_try.flatten()
    
    # Generate the C array string
    c_array = ", ".join(map(str, flattened_sample))
    
    # Determine the shape of the original sample
    original_shape = X_test_try.shape
    
    # Write to the header file
    with open(output_file, "w") as file:
        file.write("// Auto-generated header file with test data\n")
        file.write(f"#ifndef DATA_H\n#define DATA_H\n\n")
        file.write(f"#define SAMPLE_SIZE {len(flattened_sample)}\n")
        file.write(f"const float test_sample[SAMPLE_SIZE] = {{ {c_array} }};\n")
        file.write(f"// Original shape: {original_shape}\n")
        file.write("\n#endif // DATA_H\n")

# Example usage
output_file = "data.h"
write_test_sample_to_header(X_test_try, output_file)

In [18]:
from qkeras import *
import gc

def get_cnn_quantized_model(num_classes):

        gc.collect()
        keras.backend.clear_session()
    
    
        #qkeras model

        quantized_model = keras.Sequential([
                        keras.layers.Input((32,3)),
                        keras.layers.Reshape((8,4,3)),
                        #QActivation("quantized_bits(16)"),
                        QConv2D(filters=8, kernel_size = (5,5), padding = "same", kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QConv2D(filters=8, kernel_size = (5,5), padding = "same", kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        keras.layers.MaxPool2D(pool_size = 2),
                        keras.layers.Flatten(),
                        QDense(32, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.BatchNormalization(),
                        QActivation("quantized_relu(8)"),
                        QDense(num_classes, kernel_quantizer="quantized_bits(8)"),
                        keras.layers.Activation('softmax'),
                        ])

        return quantized_model
    

In [19]:
qmodel = get_cnn_quantized_model(len(label_encoder.classes_))

qmodel.compile(loss="categorical_crossentropy", optimizer='adam', metrics = ['accuracy'])

qmodel.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 8, 4, 3)           0         
                                                                 
 q_conv2d (QConv2D)          (None, 8, 4, 8)           608       
                                                                 
 batch_normalization (Batch  (None, 8, 4, 8)           32        
 Normalization)                                                  
                                                                 
 q_activation (QActivation)  (None, 8, 4, 8)           0         
                                                                 
 q_conv2d_1 (QConv2D)        (None, 8, 4, 8)           1608      
                                                                 
 batch_normalization_1 (Bat  (None, 8, 4, 8)           32        
 chNormalization)                                       

In [93]:
# Train and evaluate the quantization aware model
es = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_delta=0.0001, mode='auto', cooldown=0, min_lr=0)
                
    ]
qmodel.fit(
                  X_train,
                  y_train,
                  epochs=20,
                  validation_data=(X_validation, y_validation),
                  callbacks=[es]
              )



Epoch 1/20
214/214 [==============================] - 2s 4ms/step - loss: 2.3573 - accuracy: 0.3563 - val_loss: 1.8706 - val_accuracy: 0.5825 - lr: 0.0010
Epoch 2/20
214/214 [==============================] - 1s 2ms/step - loss: 1.4135 - accuracy: 0.7453 - val_loss: 1.1697 - val_accuracy: 0.7781 - lr: 0.0010
Epoch 3/20
214/214 [==============================] - 1s 2ms/step - loss: 0.8860 - accuracy: 0.8596 - val_loss: 0.7265 - val_accuracy: 0.8825 - lr: 0.0010
Epoch 4/20
214/214 [==============================] - 1s 3ms/step - loss: 0.5742 - accuracy: 0.9152 - val_loss: 0.5060 - val_accuracy: 0.9197 - lr: 0.0010
Epoch 5/20
214/214 [==============================] - 1s 3ms/step - loss: 0.4037 - accuracy: 0.9443 - val_loss: 0.3527 - val_accuracy: 0.9456 - lr: 0.0010
Epoch 6/20
214/214 [==============================] - 1s 3ms/step - loss: 0.2972 - accuracy: 0.9607 - val_loss: 0.2917 - val_accuracy: 0.9557 - lr: 0.0010
Epoch 7/20
214/214 [==============================] - 1s 3ms/step - lo

In [94]:
# evaluate the model on the test set
quant_loss, quant_acc = qmodel.evaluate(X_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)
qmodel.save('qat_cnn_gesture_classification_model.h5')

Quantization aware training loss:  0.040925752371549606
Quantization aware training accuracy:  0.9929824471473694


/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [95]:
# convert the QAT model to a fully quantized model using TFLite

def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(qmodel)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp4yqthdn9/assets


INFO:tensorflow:Assets written to: /tmp/tmp4yqthdn9/assets
/home/amroset/anaconda3/envs/microcontrollers/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-12-29 19:17:45.985105: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-12-29 19:17:45.985124: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-12-29 19:17:45.985230: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4yqthdn9
2024-12-29 19:17:45.988489: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-12-29 19:17:45.988500: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4yqthdn9
2024-12-29 19:17:45.997999: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bund

In [96]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant_int8_qat)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)
# Save the quantized model to disk
open("gesture_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

# Show the model size for the 8-bit quantized TFLite model
tflite_quant_in_kb = os.path.getsize('models/gesture_qat_int8.tflite') / 1024
print("TFLite Model size with 8-bit quantization: %d KB" % tflite_quant_in_kb)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
TFLite Model size with 8-bit quantization: 22 KB


In [97]:
c_model_name = 'qat8_gesture'
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))

In [24]:
# this might take a few minutes (~ 1- 2 minutes)
# if it takes longer than that, I suggest to 
# restart the runtime and try again
# if the issue still persists, restart your computer
tflite_model_quant_int8_qat_file = pathlib.Path('models/no_qkeras_gesture_qat_int8.tflite')
tflite_model_quant_int8_qat_type = "Full QAT INT8"

evaluate_model(tflite_model_quant_int8_qat_file, tflite_model_quant_int8_qat_type)
print("Full-precision model accuracy is %.4f%% (Number of test samples=%d)" % (test_accuracy * 100, len(X_train)))

Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.
Full QAT INT8 model accuracy is 97.7632% (Number of test samples=2280)
Full-precision model accuracy is 97.5000% (Number of test samples=6840)


In [23]:
# evaluate the model on the test set
tflite_model_quant_int8_qat_file = "models/no_qkeras_gesture_qat_int8.tflite"
tflite_model_quant_int8_qat_model_type = "Quantized aware training model"
tflite_model_quant_int8_qat_pred = run_tflite_model(tflite_model_quant_int8_qat_file, range(X_train.shape[0]))


Evaluated on 0 sequences.
Evaluated on 100 sequences.
Evaluated on 200 sequences.
Evaluated on 300 sequences.
Evaluated on 400 sequences.
Evaluated on 500 sequences.
Evaluated on 600 sequences.
Evaluated on 700 sequences.
Evaluated on 800 sequences.
Evaluated on 900 sequences.
Evaluated on 1000 sequences.
Evaluated on 1100 sequences.
Evaluated on 1200 sequences.
Evaluated on 1300 sequences.
Evaluated on 1400 sequences.
Evaluated on 1500 sequences.
Evaluated on 1600 sequences.
Evaluated on 1700 sequences.
Evaluated on 1800 sequences.
Evaluated on 1900 sequences.
Evaluated on 2000 sequences.
Evaluated on 2100 sequences.
Evaluated on 2200 sequences.


In [100]:
# compute the accuracy of the model
full_qat_int8_accuracy = accuracy_score(labels, tflite_model_quant_int8_qat_pred)
print('Full QAT INT8 accuracy is %.4f%% (Number of test samples=%d)' % (full_qat_int8_accuracy * 100, len(y_test)))
print('Full-precision model accuracy is %.4f%% (Number of test samples=%d)' % (test_accuracy * 100, len(y_test)))

Full QAT INT8 accuracy is 99.2544% (Number of test samples=2280)
Full-precision model accuracy is 97.9825% (Number of test samples=2280)
